## Description
In this project, you will read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport you will answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. You should only use data **from year 2018** for your analysis!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

In [ ]:
# Q1. calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

citi = cities.replace(to_replace='\[note\s\d*]',value="",regex=True)
nhl_df = nhl_df.replace(to_replace='\w*Division',value=np.NaN,regex=True).dropna()
nhl = nhl_df.replace(to_replace='\*',value='',regex=True)
nhl = nhl[nhl['year']==2018]
nhl.reset_index(drop=True,inplace=True)

def get_cityp(df):
    df['city_fst'] = df['team'].apply(lambda x: x.split(' ')[0])
    df['team_lst'] = df['team'].apply(lambda x: x.split(' ')[-1])
    df['city']= 'd'
    for i in range(df.shape[0]):
        for j in range(citi.shape[0]):
            if df['city_fst'][i] in citi['Metropolitan area'][j] and df['team_lst'][i] in citi[df['League'][i]][j]:
                df['city'][i]=citi['Metropolitan area'][j]
            elif df['team_lst'][i] in citi[df['League'][i]][j] and len(df['city'][i])<3 :
                df['city'][i]=citi['Metropolitan area'][j]
#    return print('All done! here are the first 10 entries:', df.head(10))
    df['W'] = [int(x) for x in df['W']]
    df['L'] = [int(x) for x in df['L']]
    citi_df = citi[['Metropolitan area','Population (2016 est.)[8]']]
    citi_df.rename(columns={'Metropolitan area':'city',
                         'Population (2016 est.)[8]':'pop'},inplace=True)
    citi_df['pop'] = [int(x) for x in citi_df['pop']]
    city_ttl = df.groupby('city').agg({'W':np.sum,
                        'L':np.sum})
    city_ttl.reset_index(inplace=True)
    citi_df = pd.merge(citi_df,
                 city_ttl,
                 how='inner')
    citi_df['W-L%'] = citi_df['W'] / (citi_df['W'] + citi_df['L'])
    return citi_df

def nhl_correlation():
    #df = get_cityp(nhl)
    
    population_by_region = get_cityp(nhl)['pop'] # pass in metropolitan area population from cities
    win_loss_by_region = get_cityp(nhl)['W-L%'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    
nhl_correlation()

In [ ]:
# Q2. calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

nba_df=pd.read_csv("assets/nba.csv")
nba_df.replace(to_replace='\*',value='',regex=True,inplace=True)
nba_df.replace(to_replace='\s\(\d*\)',value='',regex=True,inplace=True)
nba_df.reset_index(drop=True,inplace=True)
nba = nba_df[nba_df['year']==2018]

def nba_correlation():
    
    population_by_region = get_cityp(nba)['pop'] # pass in metropolitan area population from cities
    win_loss_by_region = get_cityp(nba)['W-L%'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [ ]:
# Q3. calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

mlb_df=pd.read_csv("assets/mlb.csv")
mlb_df.reset_index(drop=True,inplace=True)
mlb = mlb_df[mlb_df['year']==2018]

def mlb_correlation(): 
    
    population_by_region = get_cityp(mlb)['pop'] # pass in metropolitan area population from cities
    win_loss_by_region = get_cityp(mlb)['W-L%'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [ ]:
# Q4. calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

nfl_df=pd.read_csv("assets/nfl.csv")
nfl = nfl_df.replace(to_replace='[A-Z]*FC',value=np.NaN,regex=True).dropna()
nfl = nfl[nfl['year']==2018]
nfl = nfl.replace(to_replace='[\+\*]',value='',regex=True)
nfl.reset_index(drop=True,inplace=True)

def nfl_correlation(): 
    
    population_by_region = get_cityp(nfl)['pop'] # pass in metropolitan area population from cities
    win_loss_by_region = get_cityp(nfl)['W-L%'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


In [ ]:
# Q5. explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports. 
# Explore with a series of paired t-tests between all pairs of sports. Are there any sports where we can reject the null hypothesis? 
# average values where a sport has multiple teams in one region. you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate.

q5_nhl = get_cityp(nhl)[['city','W-L%']]
q5_nba = get_cityp(nba)[['city','W-L%']]
q5_mlb = get_cityp(mlb)[['city','W-L%']]
q5_nfl = get_cityp(nfl)[['city','W-L%']]

data_set = {'NFL': q5_nfl,
            'NBA': q5_nba,
            'NHL': q5_nhl,
            'MLB': q5_mlb}
sports = ['NFL', 'NBA', 'NHL', 'MLB']

def get_pvalue(k):
    pvalue = []
    for i in sports:
        test = pd.merge(data_set[k],data_set[i],
         how='inner',
         left_on='city',
         right_on='city'
         )
        p = stats.ttest_rel(test['W-L%_x'],test['W-L%_y'])[1]
        pvalue.append(p)
    return pvalue
        
    
def sports_team_performance():
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k: get_pvalue(k) for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values